In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras import Model

In [26]:
# Download dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# Add channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [27]:
# use tf.data to batch and shuffle
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).shuffle(10000).batch(32)    

In [28]:
class myModel(tf.keras.Model):
    def __init__(self):
        super (myModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = myModel()

In [29]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [30]:
# Use tf.GradientTape to train the model
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [31]:
#  Test the Model
@tf.function

def test_step(images, labels):
    # training = False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout)
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [33]:
EPOCHS = 15

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result():.4f}, '
        f'Accuracy: {train_accuracy.result() * 100:.4f}, '
        f'Test Loss: {test_loss.result():.4f}, '
        f'Test Accuracy: {test_accuracy.result() * 100:.4f}')

Epoch 1, Loss: 0.0015, Accuracy: 99.9550, Test Loss: 0.0978, Test Accuracy: 98.4900
Epoch 2, Loss: 0.0020, Accuracy: 99.9317, Test Loss: 0.0997, Test Accuracy: 98.2400
Epoch 3, Loss: 0.0022, Accuracy: 99.9233, Test Loss: 0.1094, Test Accuracy: 98.4700
Epoch 4, Loss: 0.0011, Accuracy: 99.9700, Test Loss: 0.0968, Test Accuracy: 98.6000
Epoch 5, Loss: 0.0028, Accuracy: 99.9117, Test Loss: 0.1043, Test Accuracy: 98.5100
Epoch 6, Loss: 0.0014, Accuracy: 99.9567, Test Loss: 0.1412, Test Accuracy: 98.3100
Epoch 7, Loss: 0.0019, Accuracy: 99.9400, Test Loss: 0.1144, Test Accuracy: 98.4700
Epoch 8, Loss: 0.0016, Accuracy: 99.9517, Test Loss: 0.1570, Test Accuracy: 98.1600
Epoch 9, Loss: 0.0017, Accuracy: 99.9533, Test Loss: 0.1246, Test Accuracy: 98.4000
Epoch 10, Loss: 0.0003, Accuracy: 99.9850, Test Loss: 0.1489, Test Accuracy: 98.2300
Epoch 11, Loss: 0.0010, Accuracy: 99.9750, Test Loss: 0.1229, Test Accuracy: 98.3200
Epoch 12, Loss: 0.0014, Accuracy: 99.9617, Test Loss: 0.1352, Test Accurac